In [7]:
import os
import requests

API_KEY =  os.getenv('GRAPH_API_KEY')

def fetch_pool_metadata(thegraph_api_key, pool_address):
    query = """
    {
      pools(where: { id: "%s" }) {
        id
        token0 {
          id
          symbol
          name
        }
        token1 {
          id
          symbol
          name
        }
        feeTier
        sqrtPrice
        liquidity
        tick
      }
    }
    """ % pool_address.lower()  # Ensure the address is in lowercase

    try:
        response = requests.post(
            f'https://gateway.thegraph.com/api/{thegraph_api_key}/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV',
            json={'query': query}
        )  
    except Exception as e:
        print(f"Post error occurred: {e}")
        return

    if response.status_code == 200:
        data = response.json()
        if "data" in data and "pools" in data["data"] and len(data["data"]["pools"]) > 0:
            return data["data"]["pools"][0]
        else:
            raise ValueError(f"No data found for pool address {pool_address}")
    else:
        raise ValueError(f"Failed to fetch data from The Graph: {response.status_code} - {response.text}")

# Example usage
pool_address = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"  # Example Uniswap V3 USDC/ETH pool
pool = fetch_pool_metadata(
    thegraph_api_key=API_KEY,
    pool_address=pool_address)
print(pool)

{'feeTier': '3000', 'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'liquidity': '1262989184115607311', 'sqrtPrice': '1345252239319690827966143325149066', 'tick': '194804', 'token0': {'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'name': 'USD Coin', 'symbol': 'USDC'}, 'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}}


In [10]:
print(f"Pool fee tier: {int(pool['feeTier'])*1e-6}")

Pool fee tier: 0.003


In [12]:
print(f"token0 symbol: {pool['token0']['symbol']}")

token0 symbol: USDC


In [13]:
print(f"token1 symbol: {pool['token1']['symbol']}")

token1 symbol: WETH


In [28]:
pools = [
    {'token0': 'USD', 'token1': 'ETH', 'address': '0x123'},
    {'token0': 'ETH', 'token1': 'USD', 'address': '0x456'},
    {'token0': 'ETH', 'token1': 'USD', 'address': '0x999'},
    {'token0': 'BTC', 'token1': 'USD', 'address': '0x789'},
    {'token0': 'USD', 'token1': 'BTC', 'address': '0xabc'},
    {'token0': 'USD', 'token1': 'BTC', 'address': '0xdef'},
]

In [29]:
from itertools import combinations
def find_matching_addresses(pools):
    pair_to_addresses = {}
    matching_addresses = []
    
    for pool in pools:
        # Create a pair (order doesn't matter, so we use a tuple and sort it)
        pair = tuple(sorted([pool['token0'], pool['token1']]))
        address = pool['address']
        
        if pair not in pair_to_addresses:
            pair_to_addresses[pair] = []
        
        # Add the current address to the list of addresses for this pair
        pair_to_addresses[pair].append(address)
    
    # For each token pair, generate all possible combinations of addresses
    for addresses in pair_to_addresses.values():
        if len(addresses) > 1:
            matching_addresses.extend(list(combinations(addresses, 2)))
    
    return matching_addresses

# Example usage
matches = find_matching_addresses(pools)
print("Matching addresses:", matches)

Matching addresses: [('0x123', '0x456'), ('0x123', '0x999'), ('0x456', '0x999'), ('0x789', '0xabc'), ('0x789', '0xdef'), ('0xabc', '0xdef')]


In [31]:
matching_pools = []
for addr0,addr1 in matches:
    pool_pairs = {'pool0':dict, 'pool1':dict}
    for pool in pools:
        if addr0 == pool['address']:
            pool_pairs['pool0'] = pool
        elif addr1 == pool['address']:
            pool_pairs['pool1'] = pool
    matching_pools.append(pool_pairs)

print(matching_pools)

[{'pool0': {'token0': 'USD', 'token1': 'ETH', 'address': '0x123'}, 'pool1': {'token0': 'ETH', 'token1': 'USD', 'address': '0x456'}}, {'pool0': {'token0': 'USD', 'token1': 'ETH', 'address': '0x123'}, 'pool1': {'token0': 'ETH', 'token1': 'USD', 'address': '0x999'}}, {'pool0': {'token0': 'ETH', 'token1': 'USD', 'address': '0x456'}, 'pool1': {'token0': 'ETH', 'token1': 'USD', 'address': '0x999'}}, {'pool0': {'token0': 'BTC', 'token1': 'USD', 'address': '0x789'}, 'pool1': {'token0': 'USD', 'token1': 'BTC', 'address': '0xabc'}}, {'pool0': {'token0': 'BTC', 'token1': 'USD', 'address': '0x789'}, 'pool1': {'token0': 'USD', 'token1': 'BTC', 'address': '0xdef'}}, {'pool0': {'token0': 'USD', 'token1': 'BTC', 'address': '0xabc'}, 'pool1': {'token0': 'USD', 'token1': 'BTC', 'address': '0xdef'}}]
